---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification(debug=False):
    
    # Your Code Here
    if debug:
        print(nx.info(P1_Graphs[0]), "\n")
        print(nx.info(P1_Graphs[1]), "\n")
        print(nx.info(P1_Graphs[2]), "\n")
        print(nx.info(P1_Graphs[3]), "\n")
        print(nx.info(P1_Graphs[4]), "\n")
        
        for cnt, G in enumerate(P1_Graphs):
            print("G[{}]: Avg. clu= {}\n      Avg. SPL= {}".format(
                cnt, nx.average_clustering(G), nx.average_shortest_path_length(G)))

    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']       # Your Answer Here

# graph_identification(True)

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions(debug=False):
    
    # Your Code Here
    from sklearn import preprocessing
    from sklearn import metrics
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    
    # generate dataset
    df = pd.DataFrame(index=G.nodes())

    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    
    # Assign different measurements into dataframe
    df['degree'] = pd.Series(nx.degree_centrality(G))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G))
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G))
    
    # Split the dataframe into two sub-dataframes: df_train & df_pred
    df_train = df.dropna()
    df_pred = df[np.isnan(df['ManagementSalary'])]
    
    selected_attrs = ['Department', 'degree', 'closeness', 'betweenness', 'clustering']
#     selected_attrs = ['degree', 'closeness', 'betweenness', 'clustering']
    
    X = df_train[selected_attrs]
    y = df_train['ManagementSalary']
    
    # split into training and test data w/ given training dataframe
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
   
    # Scale transformation
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    X_pred = scaler.transform(df_pred[selected_attrs])
    
    # Traing data with Logistic Regression Model
    clf = LogisticRegression()

    clf.fit(X_train, y_train)
    
    # check with training dataset with various metrics
    if debug:
        # predict the X_test
        pred1 = clf.predict(X_test)
        pred2 = clf.predict_proba(X_test)
        
        print("\nAccuracy: {}".format(metrics.accuracy_score(y_test, pred1)))
        print("\nROC_AUC: {}".format(metrics.roc_auc_score(y_test, pred2[:, 1])))
        print("\nConfusiion matrix: \n{}".format(metrics.confusion_matrix(y_test, pred1)))
        print("\nCalssification report: \n{}".format(metrics.classification_report(y_test, pred1)))
        
    # Generte result for autograder
    pred = clf.predict_proba(X_pred)
    
    rlt = pd.Series(data=pred[:, 1], index=df_pred.index)
    
    return rlt # Your Answer Here

# salary_predictions(debug=True)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [7]:
def new_connections_predictions(debug=False):
    
    # Your Code Here
    from sklearn import preprocessing
    from sklearn import metrics
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split

    # Preprocessing to add various Link Prediction measures
    # Not all of them used due to exution time limitation
    future_connections['comm_neigh'] = [len(list(
        nx.common_neighbors(G, edge[0], edge[1]))) for edge in future_connections.index]
#     future_connections['jaccard'] = [list(
#         nx.jaccard_coefficient(G, [edge]))[0][2] for edge in future_connections.index]
    future_connections['adamic_adar'] = [list(
        nx.adamic_adar_index(G, [edge]))[0][2] for edge in future_connections.index]
#     future_connections['res_alloc'] = [list(
#         nx.resource_allocation_index(G, [edge]))[0][2] for edge in future_connections.index]
    future_connections['pref_attach'] = [list(
        nx.preferential_attachment(G, [edge]))[0][2] for edge in future_connections.index]
    
    # split whole dataset into training and predict datasets via NaN in Future Connection column
    df_train = future_connections.dropna()
    df_pred = future_connections[np.isnan(future_connections['Future Connection'])]
    
    # Feature selection
    # selected_attrs = ['comm_neigh', 'jaccard', 'adamic_adar', 'res_alloc', 'pref_attach']
    selected_attrs = ['comm_neigh', 'adamic_adar', 'pref_attach']
    
    # Processing training dataset into X and y datasets
    X = df_train[selected_attrs]
    y = df_train['Future Connection']
    
    # Geneerate model model training and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Traing data with Logistic Regression Model
    clf = LogisticRegression()

    clf.fit(X_train, y_train)
    
    if debug: 
        # predict the X_test
        pred1 = clf.predict(X_test)
        pred2 = clf.predict_proba(X_test)

        print("\nAccuracy: {}".format(metrics.accuracy_score(y_test, pred1)))
        print("\nROC_AUC: {}".format(metrics.roc_auc_score(y_test, pred2[:, 1])))
        print("\nConfusiion matrix: \n{}".format(metrics.confusion_matrix(y_test, pred1)))
        print("\nCalssification report: \n{}".format(metrics.classification_report(y_test, pred1)))
    
    # Generate prediction data
    X_pred = df_pred[selected_attrs]
    
    # Generte result for autograder
    pred = clf.predict_proba(X_pred)
    
    rlt = pd.Series(data=pred[:, 1], index=df_pred.index)
    
    return rlt # Your Answer Here

# new_connections_predictions(True)


Accuracy: 0.9577162979240312

ROC_AUC: 0.9071000899915729

Confusiion matrix: 
[[66752   619]
 [ 2479  3417]]

Calssification report: 
             precision    recall  f1-score   support

        0.0       0.96      0.99      0.98     67371
        1.0       0.85      0.58      0.69      5896

avg / total       0.95      0.96      0.95     73267



(107, 348)    0.037508
(542, 751)    0.013362
(20, 426)     0.585797
(50, 989)     0.013621
(942, 986)    0.013904
(324, 857)    0.013585
(13, 710)     0.208149
(19, 271)     0.095342
(319, 878)    0.013712
(659, 707)    0.013389
(49, 843)     0.013794
(208, 893)    0.013451
(377, 469)    0.005789
(405, 999)    0.025663
(129, 740)    0.019818
(292, 618)    0.026340
(239, 689)    0.013730
(359, 373)    0.008289
(53, 523)     0.037660
(276, 984)    0.013780
(202, 997)    0.013835
(604, 619)    0.047888
(270, 911)    0.013725
(261, 481)    0.067560
(200, 450)    0.907439
(213, 634)    0.013283
(644, 735)    0.050825
(346, 553)    0.012656
(521, 738)    0.011511
(422, 953)    0.020866
                ...   
(672, 848)    0.013725
(28, 127)     0.967821
(202, 661)    0.012551
(54, 195)     0.999974
(295, 864)    0.013545
(814, 936)    0.013322
(839, 874)    0.013904
(139, 843)    0.013567
(461, 544)    0.010757
(68, 487)     0.010629
(622, 932)    0.013482
(504, 936)    0.016694
(479, 528) 